# 여러 파일 동시에

In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob

# 입력 및 출력 폴더 경로 설정
input_folder = r"\Users\kyn03\OneDrive\바탕 화면\project_file\input\합치기"
output_folder = r"\Users\kyn03\OneDrive\바탕 화면\project_file\input\빈도분석"
os.makedirs(output_folder, exist_ok=True)

# 모든 CSV 파일에 대해 반복 처리
for file_path in glob(os.path.join(input_folder, "*.csv")):
    original_file_name = os.path.splitext(os.path.basename(file_path))[0]

    # CSV 파일 읽기
    df = pd.read_csv(file_path, encoding='utf-8', engine='python').reset_index(drop=True)
    df = df.where(pd.notna(df), None)

    # 7개의 row씩 묶기
    n_values = range(len(df) // 5)

    # 선택할 row를 저장할 리스트
    selected_rows_list = []

    for n in n_values:
        indices = [0 + 5 * n, 2 + 5 * n]
        selected_rows = df.iloc[indices]
        selected_rows_list.append(selected_rows)

    # 최종 선택한 row들
    final_selected_rows = pd.concat(selected_rows_list, ignore_index=True)

    # 결과를 담을 딕셔너리 초기화
    my_dict = {}

    # final_selected_rows에서 (0, 1) 쌍으로 가져오기
    for i in range(0, len(final_selected_rows), 2):
        row_pair = final_selected_rows.iloc[i:i + 2]
        seen_aspects = set()

        for j in range(1, len(row_pair.columns)):
            if pd.notna(row_pair.iloc[1, j]) and row_pair.iloc[1, j] != "" and row_pair.iloc[1, j] != 0:
                aspect_label = row_pair.iloc[1, j][2:]
                aspect_label = aspect_label.replace('B-', '').replace('I-', '')

                if pd.notna(row_pair.iloc[0, j]) and row_pair.iloc[0, j] != "" and row_pair.iloc[0, j] != 0:
                    sentiment_label = row_pair.iloc[0, j][2:]
                    sentiment_label = sentiment_label.replace('B-', '').replace('I-', '')

                    if aspect_label in seen_aspects:
                        continue

                    seen_aspects.add(aspect_label)
                    new_dict = aspect_label + '-' + sentiment_label
                    if new_dict in my_dict:
                        my_dict[new_dict] += 1
                    else:
                        my_dict[new_dict] = 1

    # 새로운 딕셔너리를 위한 초기화
    data_dict = my_dict
    summary_dict = {}

    # 총 개수 계산
    total_count = sum(count for key, count in data_dict.items() if key != '-')

    # 데이터 정리
    for key, count in data_dict.items():
        aspect, sentiment = key.rsplit('-', 1)
        if sentiment and sentiment.strip():
            if aspect not in summary_dict:
                summary_dict[aspect] = {'긍정수': 0, '부정수': 0, '중립수': 0}

        if sentiment == '긍정':
            summary_dict[aspect]['긍정수'] += count
        elif sentiment == '부정':
            summary_dict[aspect]['부정수'] += count
        elif sentiment == '중립':
            summary_dict[aspect]['중립수'] += count

    # 최종 DataFrame 생성
    final_data = []
    for aspect, counts in summary_dict.items():
        total = counts['긍정수'] + counts['부정수'] + counts['중립수']
        final_data.append({
            'Aspect': aspect,
            '총합': total_count,
            '긍정수': counts['긍정수'],
            '부정수': counts['부정수'],
            '중립수': counts['중립수'],
        })

    # DataFrame으로 변환
    df_summary = pd.DataFrame(final_data)

    # 파일 저장
    output_file_csv = os.path.join(output_folder, f"{original_file_name}.csv")
    df_summary.to_csv(output_file_csv, index=False, encoding='utf-8-sig')

    print(f"[INFO] CSV 파일 '{output_file_csv}'로 저장 완료.")


[INFO] CSV 파일 '\Users\kyn03\OneDrive\바탕 화면\project_file\input\빈도분석\건성_가을.csv'로 저장 완료.
[INFO] CSV 파일 '\Users\kyn03\OneDrive\바탕 화면\project_file\input\빈도분석\건성_겨울.csv'로 저장 완료.
[INFO] CSV 파일 '\Users\kyn03\OneDrive\바탕 화면\project_file\input\빈도분석\건성_봄.csv'로 저장 완료.
[INFO] CSV 파일 '\Users\kyn03\OneDrive\바탕 화면\project_file\input\빈도분석\건성_여름.csv'로 저장 완료.
[INFO] CSV 파일 '\Users\kyn03\OneDrive\바탕 화면\project_file\input\빈도분석\지성_가을.csv'로 저장 완료.
[INFO] CSV 파일 '\Users\kyn03\OneDrive\바탕 화면\project_file\input\빈도분석\지성_겨울.csv'로 저장 완료.
[INFO] CSV 파일 '\Users\kyn03\OneDrive\바탕 화면\project_file\input\빈도분석\지성_봄.csv'로 저장 완료.
[INFO] CSV 파일 '\Users\kyn03\OneDrive\바탕 화면\project_file\input\빈도분석\지성_여름.csv'로 저장 완료.


## 달바

In [1]:
import pandas as pd
import numpy as np
import os

# CSV 파일 불러오기
file_path = r"\Users\kyn03\Downloads\합치기\리뷰어_3회이상.csv"  # 여기에 CSV 파일 경로를 입력하세요
original_file_name = os.path.splitext(os.path.basename(file_path))[0]

df = pd.read_csv(file_path, encoding='utf-8', engine='python').reset_index(drop=True)
df = df.where(pd.notna(df), None)  # 중복된 행은 유지하되, 결측치를 NA로 채움

# 7개의 row씩 묶기
n_values = range(len(df) // 7)  # 예를 들어, n = 0, 1, 2, 3, 4

# 선택할 row를 저장할 리스트
selected_rows_list = []

for n in n_values:
    # 선택할 인덱스 계산
    indices = [0 + 7 * n, 2 + 7 * n]  # 첫번째와 세번째 row 선택
    selected_rows = df.iloc[indices]  # 해당 row 가져오기
    selected_rows_list.append(selected_rows)  # 리스트에 추가

# 최종 선택한 row들 출력
final_selected_rows = pd.concat(selected_rows_list, ignore_index=True)

# 결과를 담을 리스트 초기화
my_dict = {}

# final_selected_rows에서 (0, 1) 쌍으로 가져오기
for i in range(0, len(final_selected_rows), 2):
    row_pair = final_selected_rows.iloc[i:i + 2]  # (i, i+1) 쌍
    seen_aspects = set()  # 이미 처리된 속성을 저장할 집합

    for j in range(1, len(row_pair.columns)):
        if pd.notna(row_pair.iloc[1, j]) and row_pair.iloc[1, j] != "" and row_pair.iloc[1, j] != 0:
            aspect_label = row_pair.iloc[1, j][2:]  # '품질/디자인/구성'
            aspect_label = aspect_label.replace('B-', '').replace('I-', '')  # B- 태그와 I- 태그 제거
        
            if pd.notna(row_pair.iloc[0, j]) and row_pair.iloc[0, j] != "" and row_pair.iloc[0, j] != 0:
                sentiment_label = row_pair.iloc[0, j][2:]  # '긍정'
                sentiment_label = sentiment_label.replace('B-', '').replace('I-', '')  # B- 태그와 I- 태그 제거

                # 속성이 이미 처리된 경우 넘어간다
                if aspect_label in seen_aspects:
                    continue

                # 속성이 처음 등장한 경우 집합에 추가하고 카운트
                seen_aspects.add(aspect_label)
                
                new_dict = aspect_label + '-' + sentiment_label
                if new_dict in my_dict:
                    my_dict[new_dict] += 1  # 존재하면 값 +1
                else:
                    my_dict[new_dict] = 1  # 존재하지 않으면 추가하고 값 1로 설정

print(my_dict)

# 새로운 딕셔너리를 위한 초기화
data_dict = my_dict
summary_dict = {}

# 총 개수 계산
total_count = sum(count for key, count in data_dict.items() if key != '-')

# 데이터 정리
for key, count in data_dict.items():
    aspect, sentiment = key.rsplit('-', 1)  # '-'를 기준으로 분리
    if sentiment and sentiment.strip():  # aspect가 공백이 아닌 경우
        if aspect not in summary_dict:
            summary_dict[aspect] = {'긍정수': 0, '부정수': 0, '중립수': 0}
    # 수치 추가
    if sentiment == '긍정':
        summary_dict[aspect]['긍정수'] += count
    elif sentiment == '부정':
        summary_dict[aspect]['부정수'] += count
    elif sentiment == '중립':
        summary_dict[aspect]['중립수'] += count

# 최종 DataFrame 생성
final_data = []
for aspect, counts in summary_dict.items():
    total = counts['긍정수'] + counts['부정수'] + counts['중립수']
    final_data.append({
        'Aspect': aspect,
        '총합': total_count,
        '긍정수': counts['긍정수'],
        '부정수': counts['부정수'],
        '중립수': counts['중립수'],
    })

# DataFrame으로 변환
df_summary = pd.DataFrame(final_data)

# CSV 파일로 저장
df_summary.to_csv(rf"\Users\kyn03\Downloads\빈도분석\{original_file_name}.csv", index=False, encoding='utf-8-sig')

# 결과 확인
print(df_summary)


{'-': 1176, '품질/디자인/구성-긍정': 398, '편의성/활용성-긍정': 436, '효과/성능/기능-긍정': 743, '사용감/착용감-긍정': 422, '사이즈/무게/개수-긍정': 52, '가격-부정': 76, '가격-긍정': 241, '사용감/착용감-부정': 57, '효과/성능/기능-부정': 48, '품질/디자인/구성-중립': 13, '편의성/활용성-': 52, '사용감/착용감-': 18, '-긍정': 62, '품질/디자인/구성-부정': 59, '제조/유통/서비스-긍정': 4, '사이즈/무게/개수-부정': 21, '사이즈/무게/개수-': 2, '사이즈/무게/개수-중립': 3, '효과/성능/기능-중립': 14, '사용감/착용감-중립': 10, '편의성/활용성-부정': 13, '품질/디자인/구성-': 24, '가격-': 14, '제조/유통/서비스-부정': 1, '효과/성능/기능-': 7, '-중립': 1, '-부정': 1, '가격-중립': 2, '편의성/활용성-중립': 1}
      Aspect    총합  긍정수  부정수  중립수
0  품질/디자인/구성  2795  398   59   13
1    편의성/활용성  2795  436   13    1
2   효과/성능/기능  2795  743   48   14
3    사용감/착용감  2795  422   57   10
4  사이즈/무게/개수  2795   52   21    3
5         가격  2795  241   76    2
6             2795   62    1    1
7  제조/유통/서비스  2795    4    1    0


## 바이오힐보

In [4]:
import pandas as pd
import numpy as np

# CSV 파일 불러오기
file_path = r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_boh.csv"  # 여기에 CSV 파일 경로를 입력하세요
df = pd.read_csv(file_path, encoding='cp949', engine='python').reset_index(drop=True)
df = df.where(pd.notna(df), None)  # 중복된 행은 유지하되, 결측치를 NA로 채움

# 7개의 row씩 묶기
n_values = range(len(df) // 7)  # 예를 들어, n = 0, 1, 2, 3, 4

# 선택할 row를 저장할 리스트
selected_rows_list = []

for n in n_values:
    # 선택할 인덱스 계산
    indices = [0 + 7 * n, 2 + 7 * n]  # 첫번째와 세번째 row 선택
    selected_rows = df.iloc[indices]  # 해당 row 가져오기
    selected_rows_list.append(selected_rows)  # 리스트에 추가

# 최종 선택한 row들 출력
final_selected_rows = pd.concat(selected_rows_list, ignore_index=True)

# 결과를 담을 리스트 초기화
my_dict = {}

# final_selected_rows에서 (0, 1) 쌍으로 가져오기
for i in range(0, len(final_selected_rows), 2):
    row_pair = final_selected_rows.iloc[i:i + 2]  # (i, i+1) 쌍
    seen_aspects = set()  # 이미 처리된 속성을 저장할 집합

    for j in range(1, len(row_pair.columns)):
        if pd.notna(row_pair.iloc[1, j]) and row_pair.iloc[1, j] != "" and row_pair.iloc[1, j] != 0:
            aspect_label = row_pair.iloc[1, j][2:]  # '품질/디자인/구성'
            aspect_label = aspect_label.replace('B-', '').replace('I-', '')  # B- 태그와 I- 태그 제거
        
            if pd.notna(row_pair.iloc[0, j]) and row_pair.iloc[0, j] != "" and row_pair.iloc[0, j] != 0:
                sentiment_label = row_pair.iloc[0, j][2:]  # '긍정'
                sentiment_label = sentiment_label.replace('B-', '').replace('I-', '')  # B- 태그와 I- 태그 제거

                # 속성이 이미 처리된 경우 넘어간다
                if aspect_label in seen_aspects:
                    continue

                # 속성이 처음 등장한 경우 집합에 추가하고 카운트
                seen_aspects.add(aspect_label)
                
                new_dict = aspect_label + '-' + sentiment_label
                if new_dict in my_dict:
                    my_dict[new_dict] += 1  # 존재하면 값 +1
                else:
                    my_dict[new_dict] = 1  # 존재하지 않으면 추가하고 값 1로 설정

print(my_dict)

# 새로운 딕셔너리를 위한 초기화
data_dict = my_dict
summary_dict = {}

# 총 개수 계산
total_count = sum(count for key, count in data_dict.items() if key != '-')

# 데이터 정리
for key, count in data_dict.items():
    aspect, sentiment = key.rsplit('-', 1)  # '-'를 기준으로 분리
    if sentiment and sentiment.strip():  # aspect가 공백이 아닌 경우
        if aspect not in summary_dict:
            summary_dict[aspect] = {'긍정수': 0, '부정수': 0, '중립수': 0}
    # 수치 추가
    if sentiment == '긍정':
        summary_dict[aspect]['긍정수'] += count
    elif sentiment == '부정':
        summary_dict[aspect]['부정수'] += count
    elif sentiment == '중립':
        summary_dict[aspect]['중립수'] += count

# 최종 DataFrame 생성
final_data = []
for aspect, counts in summary_dict.items():
    total = counts['긍정수'] + counts['부정수'] + counts['중립수']
    final_data.append({
        'Aspect': aspect,
        '총합': total_count,
        'Total Count': total_count,
        '긍정수': counts['긍정수'],
        '부정수': counts['부정수'],
        '중립수': counts['중립수'],
    })

# DataFrame으로 변환
df_summary = pd.DataFrame(final_data)

# CSV 파일로 저장
df_summary.to_csv(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_count_indivisual_boh.csv", index=False, encoding='utf-8-sig')

# 결과 확인
print(df_summary)


{'-': 1794, '품질/디자인/구성-': 42, '효과/성능/기능-긍정': 1472, '편의성/활용성-': 62, '사용감/착용감-부정': 124, '사이즈/무게/개수-부정': 33, '효과/성능/기능-부정': 170, '사용감/착용감-긍정': 600, '품질/디자인/구성-긍정': 508, '편의성/활용성-긍정': 752, '품질/디자인/구성-부정': 249, '사용감/착용감-': 57, '가격-긍정': 181, '효과/성능/기능-중립': 64, '편의성/활용성-부정': 70, '효과/성능/기능-': 19, '사이즈/무게/개수-긍정': 101, '-긍정': 86, '-중립': 5, '사이즈/무게/개수-': 8, '편의성/활용성-중립': 18, '사이즈/무게/개수-중립': 10, '사용감/착용감-중립': 23, '가격-부정': 15, '제조/유통/서비스-긍정': 2, '품질/디자인/구성-중립': 34, '-부정': 16, '가격-중립': 5, '가격-': 4}
      Aspect    총합  Total Count   긍정수  부정수  중립수
0   효과/성능/기능  4730         4730  1472  170   64
1    사용감/착용감  4730         4730   600  124   23
2  사이즈/무게/개수  4730         4730   101   33   10
3  품질/디자인/구성  4730         4730   508  249   34
4    편의성/활용성  4730         4730   752   70   18
5         가격  4730         4730   181   15    5
6             4730         4730    86   16    5
7  제조/유통/서비스  4730         4730     2    0    0


## 아벤느

In [5]:
import pandas as pd
import numpy as np

# CSV 파일 불러오기
file_path = r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_avene.csv"  # 여기에 CSV 파일 경로를 입력하세요
df = pd.read_csv(file_path, encoding='cp949', engine='python').reset_index(drop=True)
df = df.where(pd.notna(df), None)  # 중복된 행은 유지하되, 결측치를 NA로 채움

# 7개의 row씩 묶기
n_values = range(len(df) // 7)  # 예를 들어, n = 0, 1, 2, 3, 4

# 선택할 row를 저장할 리스트
selected_rows_list = []

for n in n_values:
    # 선택할 인덱스 계산
    indices = [0 + 7 * n, 2 + 7 * n]  # 첫번째와 세번째 row 선택
    selected_rows = df.iloc[indices]  # 해당 row 가져오기
    selected_rows_list.append(selected_rows)  # 리스트에 추가

# 최종 선택한 row들 출력
final_selected_rows = pd.concat(selected_rows_list, ignore_index=True)

# 결과를 담을 리스트 초기화
my_dict = {}

# final_selected_rows에서 (0, 1) 쌍으로 가져오기
for i in range(0, len(final_selected_rows), 2):
    row_pair = final_selected_rows.iloc[i:i + 2]  # (i, i+1) 쌍
    seen_aspects = set()  # 이미 처리된 속성을 저장할 집합

    for j in range(1, len(row_pair.columns)):
        if pd.notna(row_pair.iloc[1, j]) and row_pair.iloc[1, j] != "" and row_pair.iloc[1, j] != 0:
            aspect_label = row_pair.iloc[1, j][2:]  # '품질/디자인/구성'
            aspect_label = aspect_label.replace('B-', '').replace('I-', '')  # B- 태그와 I- 태그 제거
        
            if pd.notna(row_pair.iloc[0, j]) and row_pair.iloc[0, j] != "" and row_pair.iloc[0, j] != 0:
                sentiment_label = row_pair.iloc[0, j][2:]  # '긍정'
                sentiment_label = sentiment_label.replace('B-', '').replace('I-', '')  # B- 태그와 I- 태그 제거

                # 속성이 이미 처리된 경우 넘어간다
                if aspect_label in seen_aspects:
                    continue

                # 속성이 처음 등장한 경우 집합에 추가하고 카운트
                seen_aspects.add(aspect_label)
                
                new_dict = aspect_label + '-' + sentiment_label
                if new_dict in my_dict:
                    my_dict[new_dict] += 1  # 존재하면 값 +1
                else:
                    my_dict[new_dict] = 1  # 존재하지 않으면 추가하고 값 1로 설정

print(my_dict)

# 새로운 딕셔너리를 위한 초기화
data_dict = my_dict
summary_dict = {}

# 총 개수 계산
total_count = sum(count for key, count in data_dict.items() if key != '-')

# 데이터 정리
for key, count in data_dict.items():
    aspect, sentiment = key.rsplit('-', 1)  # '-'를 기준으로 분리
    if sentiment and sentiment.strip():  # aspect가 공백이 아닌 경우
        if aspect not in summary_dict:
            summary_dict[aspect] = {'긍정수': 0, '부정수': 0, '중립수': 0}
    # 수치 추가
    if sentiment == '긍정':
        summary_dict[aspect]['긍정수'] += count
    elif sentiment == '부정':
        summary_dict[aspect]['부정수'] += count
    elif sentiment == '중립':
        summary_dict[aspect]['중립수'] += count

# 최종 DataFrame 생성
final_data = []
for aspect, counts in summary_dict.items():
    total = counts['긍정수'] + counts['부정수'] + counts['중립수']
    final_data.append({
        'Aspect': aspect,
        '총합': total_count,
        'Total Count': total_count,
        '긍정수': counts['긍정수'],
        '부정수': counts['부정수'],
        '중립수': counts['중립수'],
    })

# DataFrame으로 변환
df_summary = pd.DataFrame(final_data)

# CSV 파일로 저장
df_summary.to_csv(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_count_indivisual_avene.csv", index=False, encoding='utf-8-sig')

# 결과 확인
print(df_summary)


{'효과/성능/기능-긍정': 3501, '-': 5128, '편의성/활용성-': 285, '-긍정': 229, '품질/디자인/구성-긍정': 1480, '편의성/활용성-긍정': 2878, '사용감/착용감-긍정': 2274, '가격-긍정': 935, '사용감/착용감-부정': 205, '효과/성능/기능-': 56, '가격-': 48, '사용감/착용감-중립': 37, '편의성/활용성-부정': 176, '사이즈/무게/개수-긍정': 1206, '효과/성능/기능-부정': 726, '품질/디자인/구성-': 171, '품질/디자인/구성-부정': 492, '사용감/착용감-': 109, '사이즈/무게/개수-부정': 204, '효과/성능/기능-중립': 195, '가격-부정': 142, '품질/디자인/구성-중립': 118, '제조/유통/서비스-긍정': 25, '제조/유통/서비스-부정': 6, '사이즈/무게/개수-': 47, '편의성/활용성-중립': 40, '-부정': 35, '가격-중립': 6, '사이즈/무게/개수-중립': 17, '-중립': 11, '제조/유통/서비스-': 2}
      Aspect     총합  Total Count   긍정수  부정수  중립수
0   효과/성능/기능  15656        15656  3501  726  195
1             15656        15656   229   35   11
2  품질/디자인/구성  15656        15656  1480  492  118
3    편의성/활용성  15656        15656  2878  176   40
4    사용감/착용감  15656        15656  2274  205   37
5         가격  15656        15656   935  142    6
6  사이즈/무게/개수  15656        15656  1206  204   17
7  제조/유통/서비스  15656        15656    25    6    0


# 달바

In [8]:
import pandas as pd
import numpy as np

# CSV 파일 불러오기
file_path = r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_loyal.csv"  # 여기에 CSV 파일 경로를 입력하세요
df = pd.read_csv(file_path, encoding='utf-8', engine='python').reset_index(drop=True)
df = df.where(pd.notna(df), None)  # 중복된 행은 유지하되, 결측치를 NA로 채움
# 7개의 row씩 묶기
# row를 묶기 위해 groupby 사용

n_values = range(len(df)//7)  # 예를 들어, n = 0, 1, 2, 3, 4

# 선택할 row를 저장할 리스트
selected_rows_list = []

for n in n_values:
    # 선택할 인덱스 계산
    indices = [0 + 7*n, 2 + 7*n]  # n에 따라 row 인덱스 계산
    selected_rows = df.iloc[indices]  # 해당 row 가져오기
    selected_rows_list.append(selected_rows)  # 리스트에 추가

# 최종 선택한 row들 출력
final_selected_rows = pd.concat(selected_rows_list, ignore_index=True)
# 결과를 담을 리스트 초기화
results = []

my_dict = {}

# final_selected_rows에서 (0, 1), (2, 3) 쌍으로 가져오기
for i in range(0, len(final_selected_rows), 2):
    row_pair = final_selected_rows.iloc[i:i+2]  # (i, i+1) 쌍
    for j in range(1,len(row_pair.columns)):
        if pd.notna(row_pair.iloc[0,j]) and row_pair.iloc[0,j] != "" and row_pair.iloc[0,j] != 0:
            sentiment_label = row_pair.iloc[0,j][2:]  # '긍정'
            aspect_label = row_pair.iloc[1,j][2:]  # '품질/디자인/구성'
            new_dict = aspect_label+'-'+sentiment_label
            if new_dict in my_dict:
                my_dict[new_dict] += 1  # 존재하면 값 +1
            else:
                my_dict[new_dict] = 1    # 존재하지 않으면 추가하고 값 1로 설정
                
print(my_dict)

data_dict = my_dict
# 새로운 딕셔너리를 위한 초기화
summary_dict = {}

# 총 개수 계산
total_count = sum(count for key, count in data_dict.items() if key != '-')

# 데이터 정리
for key, count in data_dict.items():
    aspect, sentiment = key.rsplit('-', 1)  # '-'를 기준으로 분리
    if sentiment and sentiment.strip():  # aspect가 공백이 아닌 경우
        if aspect not in summary_dict:
            summary_dict[aspect] = {'긍정수': 0, '부정수': 0, '중립수': 0}
    # 수치 추가
    if sentiment == '긍정':
        summary_dict[aspect]['긍정수'] += count
    elif sentiment == '부정':
        summary_dict[aspect]['부정수'] += count
    elif sentiment == '중립':
        summary_dict[aspect]['중립수'] += count

# 최종 DataFrame 생성
final_data = []
for aspect, counts in summary_dict.items():
    total = counts['긍정수'] + counts['부정수'] + counts['중립수']
    final_data.append({
        'Aspect': aspect,
        '총합': total_count,
        'Total Count': total_count,
        '긍정수': counts['긍정수'],
        '부정수': counts['부정수'],
        '중립수': counts['중립수'],
    })

# DataFrame으로 변환
df_summary = pd.DataFrame(final_data)

# CSV 파일로 저장
df_summary.to_csv(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_count_loyal.csv", index=False, encoding='utf-8-sig')

# 결과 확인
print(df_summary)

{'-': 5996}
Empty DataFrame
Columns: []
Index: []


# 바이오힐보

In [2]:
import pandas as pd
import numpy as np

# CSV 파일 불러오기
file_path = r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_boh.csv"  # 여기에 CSV 파일 경로를 입력하세요
df = pd.read_csv(file_path, encoding='cp949', engine='python').reset_index(drop=True)
df = df.where(pd.notna(df), None)  # 중복된 행은 유지하되, 결측치를 NA로 채움
# 7개의 row씩 묶기
# row를 묶기 위해 groupby 사용

n_values = range(len(df)//7)  # 예를 들어, n = 0, 1, 2, 3, 4

# 선택할 row를 저장할 리스트
selected_rows_list = []

for n in n_values:
    # 선택할 인덱스 계산
    indices = [0 + 7*n, 2 + 7*n]  # n에 따라 row 인덱스 계산
    selected_rows = df.iloc[indices]  # 해당 row 가져오기
    selected_rows_list.append(selected_rows)  # 리스트에 추가

# 최종 선택한 row들 출력
final_selected_rows = pd.concat(selected_rows_list, ignore_index=True)
# 결과를 담을 리스트 초기화
results = []

my_dict = {}

# final_selected_rows에서 (0, 1), (2, 3) 쌍으로 가져오기
for i in range(0, len(final_selected_rows), 2):
    row_pair = final_selected_rows.iloc[i:i+2]  # (i, i+1) 쌍
    for j in range(1,len(row_pair.columns)):
        if pd.notna(row_pair.iloc[0,j]) and row_pair.iloc[0,j] != "" and row_pair.iloc[0,j] != 0:
            sentiment_label = row_pair.iloc[0,j][2:]  # '긍정'
            aspect_label = row_pair.iloc[1,j][2:]  # '품질/디자인/구성'
            new_dict = aspect_label+'-'+sentiment_label
            if new_dict in my_dict:
                my_dict[new_dict] += 1  # 존재하면 값 +1
            else:
                my_dict[new_dict] = 1    # 존재하지 않으면 추가하고 값 1로 설정
                
print(my_dict)

data_dict = my_dict
# 새로운 딕셔너리를 위한 초기화
summary_dict = {}

# 총 개수 계산
total_count = sum(count for key, count in data_dict.items() if key != '-')

# 데이터 정리
for key, count in data_dict.items():
    aspect, sentiment = key.rsplit('-', 1)  # '-'를 기준으로 분리
    if sentiment and sentiment.strip():  # aspect가 공백이 아닌 경우
        if aspect not in summary_dict:
            summary_dict[aspect] = {'긍정수': 0, '부정수': 0, '중립수': 0}
    # 수치 추가
    if sentiment == '긍정':
        summary_dict[aspect]['긍정수'] += count
    elif sentiment == '부정':
        summary_dict[aspect]['부정수'] += count
    elif sentiment == '중립':
        summary_dict[aspect]['중립수'] += count

# 최종 DataFrame 생성
final_data = []
for aspect, counts in summary_dict.items():
    total = counts['긍정수'] + counts['부정수'] + counts['중립수']
    final_data.append({
        'Aspect': aspect,
        '총합': total_count,
        'Total Count': total_count,
        '긍정수': counts['긍정수'],
        '부정수': counts['부정수'],
        '중립수': counts['중립수'],
    })

# DataFrame으로 변환
df_summary = pd.DataFrame(final_data)

# CSV 파일로 저장
df_summary.to_csv(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_count_boh.csv", index=False, encoding='utf-8-sig')

# 결과 확인
print(df_summary)

{'-': 17582, '품질/디자인/구성-': 237, '효과/성능/기능-긍정': 9787, '편의성/활용성-': 335, '효과/성능/기능-부정': 1375, '사용감/착용감-부정': 953, '사이즈/무게/개수-부정': 181, '사용감/착용감-긍정': 2689, '품질/디자인/구성-긍정': 2637, '편의성/활용성-긍정': 4704, '품질/디자인/구성-부정': 1972, '-부정': 239, '사용감/착용감-': 176, '가격-긍정': 746, '-긍정': 940, '-중립': 25, '효과/성능/기능-중립': 473, '편의성/활용성-부정': 491, '효과/성능/기능-': 79, '사이즈/무게/개수-긍정': 372, '사이즈/무게/개수-': 27, '편의성/활용성-중립': 120, '사용감/착용감-중립': 140, '사이즈/무게/개수-중립': 48, '가격-부정': 56, '제조/유통/서비스-긍정': 5, '품질/디자인/구성-중립': 177, '가격-중립': 15, '가격-': 15}
      Aspect     총합  Total Count   긍정수   부정수  중립수
0   효과/성능/기능  29014        29014  9787  1375  473
1    사용감/착용감  29014        29014  2689   953  140
2  사이즈/무게/개수  29014        29014   372   181   48
3  품질/디자인/구성  29014        29014  2637  1972  177
4    편의성/활용성  29014        29014  4704   491  120
5             29014        29014   940   239   25
6         가격  29014        29014   746    56   15
7  제조/유통/서비스  29014        29014     5     0    0


# 아벤느

In [3]:
import pandas as pd
import numpy as np

# CSV 파일 불러오기
file_path = r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_avene.csv"  # 여기에 CSV 파일 경로를 입력하세요
df = pd.read_csv(file_path, encoding='cp949', engine='python').reset_index(drop=True)
df = df.where(pd.notna(df), None)  # 중복된 행은 유지하되, 결측치를 NA로 채움
# 7개의 row씩 묶기
# row를 묶기 위해 groupby 사용

n_values = range(len(df)//7)  # 예를 들어, n = 0, 1, 2, 3, 4

# 선택할 row를 저장할 리스트
selected_rows_list = []

for n in n_values:
    # 선택할 인덱스 계산
    indices = [0 + 7*n, 2 + 7*n]  # n에 따라 row 인덱스 계산
    selected_rows = df.iloc[indices]  # 해당 row 가져오기
    selected_rows_list.append(selected_rows)  # 리스트에 추가

# 최종 선택한 row들 출력
final_selected_rows = pd.concat(selected_rows_list, ignore_index=True)
# 결과를 담을 리스트 초기화
results = []

my_dict = {}

# final_selected_rows에서 (0, 1), (2, 3) 쌍으로 가져오기
for i in range(0, len(final_selected_rows), 2):
    row_pair = final_selected_rows.iloc[i:i+2]  # (i, i+1) 쌍
    for j in range(1,len(row_pair.columns)):
        if pd.notna(row_pair.iloc[0,j]) and row_pair.iloc[0,j] != "" and row_pair.iloc[0,j] != 0:
            sentiment_label = row_pair.iloc[0,j][2:]  # '긍정'
            aspect_label = row_pair.iloc[1,j][2:]  # '품질/디자인/구성'
            new_dict = aspect_label+'-'+sentiment_label
            if new_dict in my_dict:
                my_dict[new_dict] += 1  # 존재하면 값 +1
            else:
                my_dict[new_dict] = 1    # 존재하지 않으면 추가하고 값 1로 설정
                
print(my_dict)

data_dict = my_dict
# 새로운 딕셔너리를 위한 초기화
summary_dict = {}

# 총 개수 계산
total_count = sum(count for key, count in data_dict.items() if key != '-')

# 데이터 정리
for key, count in data_dict.items():
    aspect, sentiment = key.rsplit('-', 1)  # '-'를 기준으로 분리
    if sentiment and sentiment.strip():  # aspect가 공백이 아닌 경우
        if aspect not in summary_dict:
            summary_dict[aspect] = {'긍정수': 0, '부정수': 0, '중립수': 0}
    # 수치 추가
    if sentiment == '긍정':
        summary_dict[aspect]['긍정수'] += count
    elif sentiment == '부정':
        summary_dict[aspect]['부정수'] += count
    elif sentiment == '중립':
        summary_dict[aspect]['중립수'] += count

# 최종 DataFrame 생성
final_data = []
for aspect, counts in summary_dict.items():
    total = counts['긍정수'] + counts['부정수'] + counts['중립수']
    final_data.append({
        'Aspect': aspect,
        '총합': total_count,
        'Total Count': total_count,
        '긍정수': counts['긍정수'],
        '부정수': counts['부정수'],
        '중립수': counts['중립수'],
    })

# DataFrame으로 변환
df_summary = pd.DataFrame(final_data)

# CSV 파일로 저장
df_summary.to_csv(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_count_avene.csv", index=False, encoding='utf-8-sig')

# 결과 확인
print(df_summary)

{'효과/성능/기능-긍정': 33599, '효과/성능/기능-부정': 8329, '-': 89299, '편의성/활용성-': 2052, '편의성/활용성-긍정': 24159, '-긍정': 4510, '품질/디자인/구성-긍정': 9807, '사용감/착용감-긍정': 12334, '가격-긍정': 4545, '사용감/착용감-부정': 1409, '효과/성능/기능-': 278, '가격-': 179, '사용감/착용감-중립': 274, '-부정': 1083, '편의성/활용성-부정': 1525, '효과/성능/기능-중립': 1650, '사이즈/무게/개수-긍정': 5649, '품질/디자인/구성-': 1167, '품질/디자인/구성-부정': 4342, '사용감/착용감-': 429, '사이즈/무게/개수-부정': 1365, '품질/디자인/구성-중립': 722, '-중립': 180, '사이즈/무게/개수-': 276, '가격-부정': 784, '제조/유통/서비스-긍정': 121, '제조/유통/서비스-부정': 42, '편의성/활용성-중립': 402, '사이즈/무게/개수-중립': 116, '가격-중립': 17, '제조/유통/서비스-': 11}
      Aspect      총합  Total Count    긍정수   부정수   중립수
0   효과/성능/기능  121356       121356  33599  8329  1650
1    편의성/활용성  121356       121356  24159  1525   402
2             121356       121356   4510  1083   180
3  품질/디자인/구성  121356       121356   9807  4342   722
4    사용감/착용감  121356       121356  12334  1409   274
5         가격  121356       121356   4545   784    17
6  사이즈/무게/개수  121356       121356   5649  1365   116
7  제조/